In [ ]:
import psycopg2
from tabulate import tabulate

class Transaction:
    def __init__(self):
        self.items = []
        self.transaction_id = None
        

    def connect_to_database(self):
        # Function to connect to the PostgreSQL database
        connection = psycopg2.connect(
            user="postgres",
            password="",
            host="localhost",
            port="5432",
            database="COURSE"
        )
        return connection

    def create_table_if_not_exists(self):
        # Function to create the "transactions" table if it doesn't exist
        connection = self.connect_to_database()
        cursor = connection.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS transactions (
                transaction_id SERIAL PRIMARY KEY,
                item_name VARCHAR(255),
                item_qty INTEGER,
                item_price INTEGER
            );
        """)
        
        connection.commit()
        connection.close()

    def create_transaction(self):
        # Function to create a customer transaction ID
        self.create_table_if_not_exists()

        connection = self.connect_to_database()
        cursor = connection.cursor()

        cursor.execute("INSERT INTO transactions DEFAULT VALUES RETURNING transaction_id;")
        self.transaction_id = cursor.fetchone()[0]

        connection.commit()
        connection.close()

    def add_item(self, item_name, item_qty, item_price):
        # Function to add an item to the transaction
        if not self.transaction_id:
            self.create_transaction()

        self.items.append({"item_name": item_name, "item_qty": item_qty, "item_price": item_price})

    def display_transaction(self):
        # Function to display the transaction in a tabular format
        headers = ["No", "Item Name", "Quantity", "Price/Item", "Total Price", "Discount", "Amount After Discount"]
        rows = [
            [
                i + 1,
                item["item_name"],
                item["item_qty"],
                item["item_price"],
                item["item_qty"] * item["item_price"],
                self.check_discount()[1],  # Discount
                self.check_discount()[2]   # Amount After Discount
            ]
            for i, item in enumerate(self.items)
        ]

        print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))

    def check_order(self):
        # Function to check the order
        for item in self.items:
            if not item["item_name"] or item["item_qty"] <= 0 or item["item_price"] <= 0:
                return False
        return True
    
    def total_price(self):
        # Function to calculate the total purchase with discounts
        total_amount = sum(item['item_qty'] * item['item_price'] for item in self.items)
        return total_amount
    
    def disc_calculation(self, percent, amount):
        # Function to calculate a discount
        disc = int(percent * amount)
        new_amount = amount - disc
        return {"discount": disc, "amount_after_discount": new_amount}
    
    def check_discount(self):
        # Function to calculate total purchase before discount for this transaction
        total_purchase_before_discount = self.total_price()

        # Apply discounts based on total purchase
        if total_purchase_before_discount > 500000:
            discount_info = self.disc_calculation(0.10, total_purchase_before_discount)
        elif total_purchase_before_discount > 300000:
            discount_info = self.disc_calculation(0.08, total_purchase_before_discount)
        elif total_purchase_before_discount > 200000:
            discount_info = self.disc_calculation(0.05, total_purchase_before_discount)
        else:
            discount_info = {"discount": 0, "amount_after_discount": total_purchase_before_discount}

        # Return a tuple with total purchase info and discount info
        return total_purchase_before_discount, discount_info['discount'], discount_info['amount_after_discount']
    
    def find_index(self, name):    
        # Function for searching index of an item 
        for index, item in enumerate(self.items):
            if item["item_name"].lower() == name.lower():
                return index
        return -1

    def update_item_name(self, item_index, new_name):
        # Function to update the item name
            old_name = self.items[item_index]["item_name"]
            found_index = self.find_index(old_name)
            if found_index != -1:
                self.items[item_index]["item_name"] = new_name
                print(f'{old_name.title()} changed to {new_name.title()}')
            else:
                print(f'Can\'t found {old_name.title()} in cart. Please input another item name')

    def update_item_qty(self, item_index, new_qty):
        # Function to update the item quantity
        name = self.items[item_index]["item_name"]
        item_index = self.find_index(name)
        if item_index != -1:
            try:
                # Error handling for casting new_qty from string to integer
                int_new_qty = int(new_qty)
                
                # Raise exception if new_qty is smaller than 0
                if int_new_qty < 0:
                    raise Exception
                
                self.items[item_index]["item_qty"] = int_new_qty
                print(f'{name.title()} quantity changed to {new_qty}')
            except ValueError:
                print("Quantity must be an integer.")
            except Exception:
                print("Quantity must not be smaller than 0!")
        else:
            print(f'Can\'t found {name.title()} in cart. Please input another item name')

    def update_item_price(self, item_index, new_price):
        # Function to update the item price
        name = self.items[item_index]["item_name"]
        item_index = self.find_index(name)
        if item_index != -1:
            try:
                # Error handling for casting new_price from string to integer
                int_new_price = int(new_price)
                
                # Raise exception if new_price is smaller than 0
                if int_new_price < 0:
                    raise Exception
                
                self.items[item_index]["item_price"] = int_new_price
                print(f'{name.title()} price changed to {new_price}')
            except ValueError:
                print("Price must be an integer.")
            except Exception:
                print("Price must not be smaller than 0!")
        else:
            print(f'Can\'t found {name.title()} in cart. Please input another item name')

    def delete_item(self, item_index):
        # Function to delete an item
        name = self.items[item_index]["item_name"]
        item_index = self.find_index(name)

        if item_index != -1:
            del self.items[item_index]
            print(f'{name.title()} is successfully removed from cart.')
        else:
            print(f'Can\'t found {name.title()} in cart. Please input another item name')

    def reset_transaction(self):
        # Function to reset the transaction
        self.transaction_id = None
        self.items = []
        print("Transaction reset. Start a new transaction.")

    def start_transaction(self,all_transaction):
        while True:
            trnset_123 = Transaction()

            try:
                item_name = input("Enter item name: ")
                item_qty = int(input("Enter item quantity: "))
                item_price = int(input("Enter item price: "))

                if item_qty <= 0 or item_price <= 0:
                    raise ValueError("Input Data Error Occurred")

                trnset_123.add_item(item_name, item_qty, item_price)
                trnset_123.display_transaction()

                # Append the current transaction to the list
                all_transactions.append(trnset_123)

            except ValueError as e:
                print(e)

            # Prompt user until valid input is received
            while True:
                another_transaction = input("Do you want to enter another transaction? (yes/no): ").lower()
                if another_transaction in {'yes', 'no'}:
                    break
                else:
                    print("Invalid input. Please enter 'yes' or 'no'.")

            if another_transaction == 'no':
                break
        
    def check_cart(self, all_transactions):
        # Check the current cart
        print("\nYour current cart:")
        headers = ["No", "Item Name", "Quantity", "Price/Item", "Total Price", "Discount", "Amount After Discount"]
        rows = []

        for i, transaction in enumerate(all_transactions, start=1):
            for j, item in enumerate(transaction.items, start=1):
                # Use check_discount() to get discount and amount after discount
                discount, amount_after_discount = transaction.check_discount()[1], transaction.check_discount()[2]
                rows.append([
                f"{i}",
                item["item_name"],
                item["item_qty"],
                item["item_price"],
                item["item_qty"] * item["item_price"],
                discount,
                amount_after_discount
            ])

        print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))

    def display_updated_cart(self):
        # Function to display the updated cart after each update or after delete
        headers = ["No", "Item Name", "Quantity", "Price/Item", "Total Price", "Discount", "Amount After Discount"]
        rows = [
            [
                i + 1,
                item["item_name"],
                item["item_qty"],
                item["item_price"],
                item["item_qty"] * item["item_price"],
                self.check_discount()[1],  # Discount
                self.check_discount()[2]   # Amount After Discount
            ]
            for i, item in enumerate(self.items)
        ]

        print("\nUpdated Cart:")
        print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))

    def check_status(self,all_transactions):
        while True:
            action = input("What do you want to do? (update/delete/reset/finish): ").lower()

            if action == 'update':
                self.check_cart(all_transactions)
                try:
                    item_index = int(input("Enter the Number of the item you want to update: "))
                    if 1 <= item_index <= len(all_transactions):
                        pass
                    else:
                        print("Invalid number. Please enter a valid number.")
                        continue
                except ValueError:
                    print("Invalid input. Please enter a valid integer number.")
                    continue

                update_type = input("What do you want to update? (name/quantity/price): ").lower()

                if update_type == 'name':
                    new_name = input("Enter the new name: ")
                    all_transactions[item_index - 1].update_item_name(item_index - 1, new_name)
                    all_transactions[item_index - 1].display_updated_cart()
                elif update_type == 'quantity':
                    new_qty = int(input("Enter the new quantity: "))
                    all_transactions[item_index - 1].update_item_qty(item_index - 1, new_qty)
                    all_transactions[item_index - 1].display_updated_cart()
                elif update_type == 'price':
                    new_price = int(input("Enter the new price: "))
                    all_transactions[item_index - 1].update_item_price(item_index - 1, new_price)
                    all_transactions[item_index - 1].display_updated_cart()
                else:
                    print("Invalid update type. Please enter 'name', 'quantity', or 'price'.")

            elif action == 'delete':
                self.check_cart(all_transactions)
                try:
                    item_index = int(input("Enter the Number of the item you want to delete: "))
                    if 1 <= item_index <= len(all_transactions):
                        pass
                    else:
                        print("Invalid number. Please enter a valid number.")
                        continue
                except ValueError:
                    print("Invalid input. Please enter a valid integer number.")
                    continue

                all_transactions[item_index - 1].delete_item(item_index - 1)
                all_transactions[item_index - 1].check_cart(all_transactions)

            elif action == 'reset':
                self.reset_transaction()

            elif action == 'finish':
                self.check_out()
                break

            else:
                print("Invalid action. Please enter 'update', 'delete', 'reset', or 'finish'.")

    def check_out(self):
        print("\n\n" + "=" * 35, " ORDER COMPLETED ", "=" * 35)
        # Summary total purchase
        total_purchase_info, total_discount, total_purchase_after_discount = 0, 0, 0
        for trnset in all_transactions:
            purchase, discount, after_discount = trnset.check_discount()
            total_purchase_info += purchase
            total_discount += discount
            total_purchase_after_discount += after_discount

        # Print the summary in tabular format
        summary_headers = ["Total Purchase before Discount", "Total Discount", "Total Purchase after Discount"]
        summary_data = [[total_purchase_info, total_discount, total_purchase_after_discount]]

        print("\nSummary of all transactions:")
        print(tabulate(summary_data, headers=summary_headers, tablefmt="fancy_grid"))
        print("\n\n" + "=" * 35, " THANK YOU FOR SHOPPING HERE ", "=" * 35)
        return True
        
            
# Main flow
all_transactions = []
transaction_instance = Transaction()

while True:
    transaction_instance.start_transaction(all_transactions)
    
    transaction_instance.check_cart(all_transactions)
    transaction_instance.check_status(all_transactions)



Enter item name: Noodle
Enter item quantity: 5
Enter item price: 3000
╒══════╤═════════════╤════════════╤══════════════╤═══════════════╤════════════╤═════════════════════════╕
│   No │ Item Name   │   Quantity │   Price/Item │   Total Price │   Discount │   Amount After Discount │
╞══════╪═════════════╪════════════╪══════════════╪═══════════════╪════════════╪═════════════════════════╡
│    1 │ Noodle      │          5 │         3000 │         15000 │          0 │                   15000 │
╘══════╧═════════════╧════════════╧══════════════╧═══════════════╧════════════╧═════════════════════════╛
Do you want to enter another transaction? (yes/no): yes
Enter item name: Tempeh
Enter item quantity: 3
Enter item price: 5000
╒══════╤═════════════╤════════════╤══════════════╤═══════════════╤════════════╤═════════════════════════╕
│   No │ Item Name   │   Quantity │   Price/Item │   Total Price │   Discount │   Amount After Discount │
╞══════╪═════════════╪════════════╪══════════════╪════════════